In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("pyspark")
sc = SparkContext(conf=conf)

In [2]:
#btc_raw = sc.textFile("graph_datasets/soc-sign-bitcoinalpha.csv")
#btc_raw.take(10)
#btc = btc_raw.map(lambda x: x.split(",")).map(lambda x: (int(x[0]), int(x[1]))).flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))

In [9]:
btc_raw = sc.parallelize([(0,1), (1,2), (2,5), (5,8), (7,8), (3,7), (3,4), (3,6), (10,11), (10,12), (12,13)])
G = btc_raw.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))

In [25]:
def Min_Selection_Step(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], (x[1] | {x[0]})))
    
    #Broadcasting
    v_min_bc = sc.broadcast(dict(v_min.collect()))
    addEdge = NN_G_u.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    print("a", addEdge.collect())
    #Without broadcasting
    #addEdge = NN_G_u.join(v_min).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    H = addEdge.groupByKey().map(lambda x: (x[0], set(x[1])))
    print("H",H.collect())
    return H

def Pruning_Step(H, T):
    
    #minimum node of the neighborhood: shared for following parts
    v_min = H.map(lambda x: (x[0], min(x[1])))
    v_min_bc = sc.broadcast(dict(v_min.collect())) #Broadcasting v_min
    
    #---------------G construction-------------------
    H_filtered = H.filter(lambda x: len(x[1]) > 1)
    NN_H_u = H_filtered.map(lambda x: (x[0], x[1] - {min(x[1])} ))
    #With Broadcasting
    addEdge2=NN_H_u.map(lambda x:(x[0],(x[1],v_min_bc.value[x[0]]))).flatMap(lambda x:[(x[1][1],y) for y in x[1][0]])
    #Without broadcasting
    #addEdge2 = NN_H_u.join(v_min).flatMap(lambda x: [(x[1][1], y) for y in x[1][0]])
    G = addEdge2.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
    
    #---------------Tree construction--------------
    deactiveNodes = H.filter(lambda x: x[0] not in x[1]).map(lambda x: (x[0], None))
    #Without broadcasting
    #addEdge3 = deactiveNodes.join(v_min).map(lambda x: (x[1][1], x[0]))
    #With Broadcasting
    addEdge3 = deactiveNodes.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).map(lambda x: (x[1][1], x[0]))
    T = T.union(addEdge3)

    return [G, T]

#Finding seeds
def findSeeds(T): 
    keys = T
    values = T.map(lambda x:(x[1], x[0]))
    return keys.subtractByKey(values).map(lambda x: x[0]).distinct()

def findSeeds1(T):
    keys = T.keys().distinct().map(lambda x:(x,1))
    values = T.values().distinct().map(lambda x:(x,1))
    return keys.subtractByKey(values).map(lambda x: x[0])

def findSeeds2(T):
    T_inv = T.map(lambda x:(x[1], x[0]))
    A = T.keys().distinct().map(lambda x:(x,1))        #Each distinct is a reduceByKey
    B = T_inv.keys().distinct().map(lambda x:(x,1))
    return A.leftOuterJoin(B).filter(lambda x: not x[1][1]).map(lambda x:x[0])

In [26]:
def Cracker(G):
    n = 0
    T = sc.parallelize([])
    while G.take(1):
        n += 1
        H = Min_Selection_Step(G)
        G, T = Pruning_Step(H, T)
    
    return findSeeds(T)

In [27]:
#Cracker with findSeeds
Cracker(G).collect()

a [(0, 0), (1, 0), (8, 5), (5, 5), (7, 5), (3, 3), (4, 3), (10, 10), (12, 10), (13, 10), (0, 0), (1, 0), (2, 0), (8, 2), (2, 2), (5, 2), (12, 12), (13, 12), (1, 1), (2, 1), (5, 1), (3, 3), (6, 3), (10, 10), (11, 10), (12, 10), (8, 3), (3, 3), (7, 3), (3, 3), (4, 3), (6, 3), (7, 3), (10, 10), (11, 10)]
H [(0, {0}), (8, {2, 3, 5}), (4, {3}), (12, {10, 12}), (1, {0, 1}), (5, {1, 2, 5}), (13, {10, 12}), (10, {10}), (2, {0, 1, 2}), (6, {3}), (7, {3, 5}), (3, {3}), (11, {10})]
a [(10, 10), (12, 10), (0, 0), (1, 0), (2, 0), (1, 1), (2, 1), (3, 1), (5, 1), (0, 0), (1, 0), (2, 0), (5, 0), (0, 0), (1, 0), (2, 0), (3, 0), (5, 0), (10, 10), (12, 10), (2, 2), (3, 2), (5, 2)]
H [(12, {10}), (0, {0}), (1, {0, 1}), (5, {0, 1, 2}), (10, {10}), (2, {0, 1, 2}), (3, {0, 1, 2})]
a [(0, 0), (1, 0), (2, 0), (0, 0), (1, 0), (0, 0), (2, 0)]
H [(0, {0}), (1, {0}), (2, {0})]


[0, 10]

In [41]:
def Min_Selection_Step(G): #dictionary format RDD
    v_min = G.map(lambda x: (x[0], min(x[1][0] | {x[0]})))
    NN_G_u = G.map(lambda x: (x[0], (x[1][0] | {x[0]}, x[1][1])))
    
    #Broadcasting
    v_min_bc = sc.broadcast(dict(v_min.collect()))
    addEdge = NN_G_u.map(lambda x: (x[0], (x[1][0], v_min_bc.value[x[0]], x[1][1])))
    addEdge1 = addEdge.flatMap(lambda x: [(y, (x[1][1], x[1][2])) for y in x[1][0]])
    print(addEdge1.collect())
    #Without broadcasting
    #addEdge = NN_G_u.join(v_min).flatMap(lambda x: [(y, x[1][1]) for y in x[1][0]])
    
    H = addEdge1.groupByKey().map(lambda x: (x[0], (list(x[1]))))

    return H

def Pruning_Step(H, T):
    
    #minimum node of the neighborhood: shared for following parts
    v_min = H.map(lambda x: (x[0], min(x[1][0])))
    v_min_bc = sc.broadcast(dict(v_min.collect())) #Broadcasting v_min
    
    #---------------G construction-------------------
    H_filtered = H.filter(lambda x: len(x[1][0]) > 1)
    NN_H_u = H_filtered.map(lambda x: (x[0], x[1][0] - {min(x[1][0])} ))
    #With Broadcasting
    addEdge2=NN_H_u.map(lambda x:(x[0],(x[1],v_min_bc.value[x[0]]))).flatMap(lambda x:[(x[1][1],y) for y in x[1][0]])
    #Without broadcasting
    #addEdge2 = NN_H_u.join(v_min).flatMap(lambda x: [(x[1][1], y) for y in x[1][0]])
    G = addEdge2.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], set(x[1])))
    
    #---------------Tree construction--------------
    deactiveNodes = H.filter(lambda x: x[0] not in x[1]).map(lambda x: (x[0], None))
    #Without broadcasting
    #addEdge3 = deactiveNodes.join(v_min).map(lambda x: (x[1][1], x[0]))
    #With Broadcasting
    addEdge3 = deactiveNodes.map(lambda x: (x[0], (x[1], v_min_bc.value[x[0]]))).map(lambda x: (x[1][1], x[0]))
    T = T.union(addEdge3)

    return [G, T]

#Finding seeds
def findSeeds(T): 
    keys = T
    values = T.map(lambda x:(x[1], x[0]))
    return keys.subtractByKey(values).map(lambda x: x[0]).distinct()

def findSeeds1(T):
    keys = T.keys().distinct().map(lambda x:(x,1))
    values = T.values().distinct().map(lambda x:(x,1))
    return keys.subtractByKey(values).map(lambda x: x[0])

def findSeeds2(T):
    T_inv = T.map(lambda x:(x[1], x[0]))
    A = T.keys().distinct().map(lambda x:(x,1))        #Each distinct is a reduceByKey
    B = T_inv.keys().distinct().map(lambda x:(x,1))
    return A.leftOuterJoin(B).filter(lambda x: not x[1][1]).map(lambda x:x[0])

In [36]:
btc_raw = sc.parallelize([(0,1), (1,2), (2,5), (5,8), (7,8), (3,7), (3,4), (3,6), (10,11), (10,12), (12,13)])
G2 = btc_raw.flatMap(lambda x: [x, (x[1], x[0])]).groupByKey().map(lambda x: (x[0], (set(x[1]), True)))

In [42]:
H2 = Min_Selection_Step(G2)
H2.collect()

[(0, (0, True)), (1, (0, True)), (8, (5, True)), (5, (5, True)), (7, (5, True)), (3, (3, True)), (4, (3, True)), (10, (10, True)), (12, (10, True)), (13, (10, True)), (0, (0, True)), (1, (0, True)), (2, (0, True)), (8, (2, True)), (2, (2, True)), (5, (2, True)), (12, (12, True)), (13, (12, True)), (1, (1, True)), (2, (1, True)), (5, (1, True)), (3, (3, True)), (6, (3, True)), (10, (10, True)), (11, (10, True)), (12, (10, True)), (8, (3, True)), (3, (3, True)), (7, (3, True)), (3, (3, True)), (4, (3, True)), (6, (3, True)), (7, (3, True)), (10, (10, True)), (11, (10, True))]


[(0, [(0, True), (0, True)]),
 (8, [(5, True), (2, True), (3, True)]),
 (4, [(3, True), (3, True)]),
 (12, [(10, True), (12, True), (10, True)]),
 (1, [(0, True), (0, True), (1, True)]),
 (5, [(5, True), (2, True), (1, True)]),
 (13, [(10, True), (12, True)]),
 (10, [(10, True), (10, True), (10, True)]),
 (2, [(0, True), (2, True), (1, True)]),
 (6, [(3, True), (3, True)]),
 (7, [(5, True), (3, True), (3, True)]),
 (3, [(3, True), (3, True), (3, True), (3, True)]),
 (11, [(10, True), (10, True)])]